In [1]:
# insurance_ai_agent/
# │
# ├─ venv/                     # virtual environment
# ├─ data/
# │   ├─ raw/                  # raw scraped data or API responses
# │   └─ processed/            # cleaned datasets
# ├─ notebooks/
# │   └─ 01_basic_data_collection.ipynb
# ├─ src/
# │   ├─ __init__.py
# │   ├─ config.py             # API keys, settings (DON'T commit real keys)
# │   ├─ collectors/           # code to get data from web
# │   │   └─ web_search.py
# │   └─ analysis/             # code for analysis and SWOT later
# │       └─ sentiment.py
# └─ requirements.txt          # library list


In [3]:
import os

# Create a folder
os.makedirs("insurance_ai_agent", exist_ok=True)

# Move into the folder
os.chdir("insurance_ai_agent")

os.getcwd()


'/Users/udaypandey/Desktop/Rag Model /Rag Agent /insurance_ai_agent'

In [4]:
!python -m venv venv


In [5]:
# installing librarries 

In [7]:
!pip install openai jupyter pandas requests


In [8]:
# creating folder structure 

In [9]:
import os

folders = [
    "data/raw",
    "data/processed",
    "src/collectors",
    "src/analysis",
    "notebooks"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Project structure created!")


Project structure created!


In [10]:
with open("notebooks/01_test_agent_browser.ipynb", "w") as f:
    f.write("")


In [12]:
# open ai browser installation 

In [14]:
# creating agaent 1 

In [15]:
import sys
print("Python executable:", sys.executable)

# Uninstall old openai
%pip uninstall -y openai

# Install the latest OpenAI SDK (supports new APIs)
%pip install "openai>=1.40.0"

# Install the Agents SDK
%pip install "openai-agents"


Python executable: /opt/anaconda3/bin/python
Found existing installation: openai 2.9.0
Uninstalling openai-2.9.0:
  Successfully uninstalled openai-2.9.0
Note: you may need to restart the kernel to use updated packages.
  Using cached openai-2.9.0-py3-none-any.whl.metadata (29 kB)
Using cached openai-2.9.0-py3-none-any.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
import openai
print(openai.__version__)


2.9.0


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ")


In [ ]:
from agents import Agent, Runner

test_agent = Agent(
    name="Test agent",
    instructions="You are a friendly assistant. Answer in one short sentence.",
    model="gpt-4.1-mini",  # or "gpt-4.1" if available
)


In [ ]:
result = await Runner.run(
    test_agent,
    "Say hello in one short sentence."
)

print(result.final_output)


In [ ]:
# Creating Agent 1 With prompts - Insurance Review Agent

In [69]:
from agents import Agent, Runner, WebSearchTool

insurance_agent = Agent(
    name="Insurance Claim Experience Researcher (JSON)",
    instructions=(
        "You are an insurance market research assistant focused on REAL USER CLAIM EXPERIENCES.\n\n"
        "When given an insurance company and product, you use web search to find posts and discussions "
        "from real customers describing:\n"
        " - How their claim was processed\n"
        " - Whether it was approved or denied\n"
        " - Delays, documentation issues, or confusion\n"
        " - Interactions with customer service during the claim\n\n"
        "PRIORITIZE sources like:\n"
        " - Reddit\n"
        " - Quora\n"
        " - Twitter (X)\n"
        " - TikTok transcripts or summaries (if available via web search)\n"
        " - Other forums and Q&A/social platforms where individuals share personal experiences\n\n"
        "You may also use review sites and blogs, BUT only as secondary sources and only when "
        "they contain first-person experiences about claims.\n\n"
        "You MUST respond ONLY in valid JSON with this exact structure:\n\n"
        "{\n"
        '  \"company\": string,\n'
        '  \"product\": string,\n'
        '  \"claim_difficulties\": [string, ...],\n'
        '  \"claim_smooth_experiences\": [string, ...],\n'
        '  \"common_complaints\": [string, ...],\n'
        '  \"major_positives\": [string, ...],\n'
        '  \"summary\": string\n'
        "}\n\n"
        "Where:\n"
        " - \"claim_difficulties\" are specific recurring issues people faced when filing or getting claims paid.\n"
        " - \"claim_smooth_experiences\" are examples/patterns where claims went well (fast, fair, easy).\n"
        " - \"common_complaints\" can include other recurring pain points mentioned in user posts.\n"
        " - \"major_positives\" can include trust, fairness, fast payouts, or anything users praise.\n\n"
        "Rules:\n"
        " - Do NOT wrap the JSON in ``` or any code block.\n"
        " - Do NOT add markdown, labels, or explanations.\n"
        " - Output ONLY the raw JSON object."
    ),
    tools=[WebSearchTool()],
    model="gpt-4.1-mini",  # or "gpt-4.1" if available
)


In [71]:
# Telling the agent to write in JSON

In [73]:
from agents import Agent, Runner, WebSearchTool

insurance_agent = Agent(
    name="Insurance Review Agent (JSON)",
    instructions=(
        "You are an insurance market research assistant. "
        "When given an insurance company and product, you use web search "
        "to find customer reviews, complaints, and comparisons. "
        "You MUST respond ONLY in valid JSON, with this exact structure:\n\n"
        "{\n"
        '  "company": string,\n'
        '  "product": string,\n'
        '  "common_complaints": [string, ...],\n'
        '  "major_positives": [string, ...],\n'
        '  "summary": string\n'
        "}\n\n"
        "Rules:\n"
        " - Do NOT wrap the JSON in ``` or any code block.\n"
        " - Do NOT add markdown, labels, or explanations.\n"
        " - Output ONLY the raw JSON object."
    ),
    tools=[WebSearchTool()],
    model="gpt-4.1-mini",
)


In [75]:
company = "Aetna"
product = "Silver PPO health insurance plan"

query = (
    f"Analyze customer feedback for {company}'s {product}. "
    "Find common complaints, major positives, and overall summary."
)

result = await Runner.run(insurance_agent, query)
raw_output = result.final_output
print("RAW OUTPUT:\n", raw_output)

# Now either:
# 1) Try direct:
# product_data = json.loads(raw_output)

# or 2) Use the safe slice version in case it still misbehaves:
start = raw_output.find("{")
end = raw_output.rfind("}") + 1
json_str = raw_output[start:end]
product_data = json.loads(json_str)

product_data


RAW OUTPUT:
 {
  "company": "Aetna",
  "product": "Silver PPO health insurance plan",
  "common_complaints": [
    "High premiums and deductibles",
    "Denied claims and coverage for necessary medical services",
    "Inefficient customer service with long wait times",
    "Limited provider networks, especially in rural areas",
    "Out-of-network care can be expensive"
  ],
  "major_positives": [
    "Comprehensive coverage options",
    "Access to a wide network of providers",
    "Preventive care services and wellness programs",
    "Financial stability with strong industry ratings",
    "Digital tools and resources for members"
  ],
  "summary": "Aetna's Silver PPO health insurance plan offers comprehensive coverage with access to a broad provider network and preventive care services. However, customers have reported high premiums and deductibles, denied claims, and inefficient customer service. While Aetna maintains financial stability and provides valuable wellness programs, pote

{'company': 'Aetna',
 'product': 'Silver PPO health insurance plan',
 'common_complaints': ['High premiums and deductibles',
  'Denied claims and coverage for necessary medical services',
  'Inefficient customer service with long wait times',
  'Limited provider networks, especially in rural areas',
  'Out-of-network care can be expensive'],
 'major_positives': ['Comprehensive coverage options',
  'Access to a wide network of providers',
  'Preventive care services and wellness programs',
  'Financial stability with strong industry ratings',
  'Digital tools and resources for members'],
 'summary': "Aetna's Silver PPO health insurance plan offers comprehensive coverage with access to a broad provider network and preventive care services. However, customers have reported high premiums and deductibles, denied claims, and inefficient customer service. While Aetna maintains financial stability and provides valuable wellness programs, potential members should weigh these benefits against th

In [77]:
product_data


{'company': 'Aetna',
 'product': 'Silver PPO health insurance plan',
 'common_complaints': ['High premiums and deductibles',
  'Denied claims and coverage for necessary medical services',
  'Inefficient customer service with long wait times',
  'Limited provider networks, especially in rural areas',
  'Out-of-network care can be expensive'],
 'major_positives': ['Comprehensive coverage options',
  'Access to a wide network of providers',
  'Preventive care services and wellness programs',
  'Financial stability with strong industry ratings',
  'Digital tools and resources for members'],
 'summary': "Aetna's Silver PPO health insurance plan offers comprehensive coverage with access to a broad provider network and preventive care services. However, customers have reported high premiums and deductibles, denied claims, and inefficient customer service. While Aetna maintains financial stability and provides valuable wellness programs, potential members should weigh these benefits against th

In [79]:
from agents import Agent, Runner

swot_agent = Agent(
    name="SWOT Generator",
    instructions=(
        "You are an insurance strategy analyst. "
        "Given structured information about an insurance product "
        "(company, product, complaints, positives, summary), "
        "you create a SWOT analysis. "
        "Always respond in this JSON structure:\n\n"
        "{\n"
        '  "company": string,\n'
        '  "product": string,\n'
        '  "strengths": [string, ...],\n'
        '  "weaknesses": [string, ...],\n'
        '  "opportunities": [string, ...],\n'
        '  "threats": [string, ...]\n'
        "}\n\n"
        "Do not include any extra text."
    ),
    model="gpt-4.1-mini",
)


In [81]:
swot_prompt = (
    "Create a SWOT analysis from the following JSON data about an insurance product:\n\n"
    f"{json.dumps(product_data, indent=2)}"
)
swot_prompt


'Create a SWOT analysis from the following JSON data about an insurance product:\n\n{\n  "company": "Aetna",\n  "product": "Silver PPO health insurance plan",\n  "common_complaints": [\n    "High premiums and deductibles",\n    "Denied claims and coverage for necessary medical services",\n    "Inefficient customer service with long wait times",\n    "Limited provider networks, especially in rural areas",\n    "Out-of-network care can be expensive"\n  ],\n  "major_positives": [\n    "Comprehensive coverage options",\n    "Access to a wide network of providers",\n    "Preventive care services and wellness programs",\n    "Financial stability with strong industry ratings",\n    "Digital tools and resources for members"\n  ],\n  "summary": "Aetna\'s Silver PPO health insurance plan offers comprehensive coverage with access to a broad provider network and preventive care services. However, customers have reported high premiums and deductibles, denied claims, and inefficient customer service

In [83]:
swot_result = await Runner.run(swot_agent, swot_prompt)
swot_raw = swot_result.final_output
print("SWOT RAW:\n", swot_raw)

swot_data = json.loads(swot_raw)
swot_data


SWOT RAW:
 {
  "company": "Aetna",
  "product": "Silver PPO health insurance plan",
  "strengths": [
    "Comprehensive coverage options",
    "Access to a wide network of providers",
    "Preventive care services and wellness programs",
    "Financial stability with strong industry ratings",
    "Digital tools and resources for members"
  ],
  "weaknesses": [
    "High premiums and deductibles",
    "Denied claims and coverage for necessary medical services",
    "Inefficient customer service with long wait times",
    "Limited provider networks, especially in rural areas",
    "Expensive out-of-network care"
  ],
  "opportunities": [
    "Expand provider network in rural areas to attract more customers",
    "Improve customer service efficiency to enhance customer satisfaction",
    "Leverage digital tools to streamline claims and coverage processes",
    "Develop more affordable premium options or deductible plans"
  ],
  "threats": [
    "Competitive insurance plans offering lower 

{'company': 'Aetna',
 'product': 'Silver PPO health insurance plan',
 'strengths': ['Comprehensive coverage options',
  'Access to a wide network of providers',
  'Preventive care services and wellness programs',
  'Financial stability with strong industry ratings',
  'Digital tools and resources for members'],
 'weaknesses': ['High premiums and deductibles',
  'Denied claims and coverage for necessary medical services',
  'Inefficient customer service with long wait times',
  'Limited provider networks, especially in rural areas',
  'Expensive out-of-network care'],
 'opportunities': ['Expand provider network in rural areas to attract more customers',
  'Improve customer service efficiency to enhance customer satisfaction',
  'Leverage digital tools to streamline claims and coverage processes',
  'Develop more affordable premium options or deductible plans'],
 'threats': ['Competitive insurance plans offering lower premiums and better customer service',
  'Customer dissatisfaction lea

In [84]:
# Combining both the agents to get clean function

In [87]:
async def analyze_product(company: str, product: str):
    # Step 1 — research
    query = (
        f"Analyze customer feedback for {company}'s {product}. "
        "Find common complaints, major positives, and overall summary."
    )

    result = await Runner.run(insurance_agent, query)
    raw_output = result.final_output
    
    # extract JSON safely
    start = raw_output.find("{")
    end = raw_output.rfind("}") + 1
    research_data = json.loads(raw_output[start:end])
    
    # Step 2 — SWOT analysis
    swot_prompt = (
        "Create a SWOT analysis from the following JSON data:\n\n"
        f"{json.dumps(research_data, indent=2)}"
    )

    swot_result = await Runner.run(swot_agent, swot_prompt)
    swot_raw = swot_result.final_output
    
    start = swot_raw.find("{")
    end = swot_raw.rfind("}") + 1
    swot_data = json.loads(swot_raw[start:end])
    
    return {
        "research": research_data,
        "swot": swot_data
    }


In [89]:
analysis = await analyze_product("Aetna", "Silver PPO health insurance plan")
analysis


{'research': {'company': 'Aetna',
  'product': 'Silver PPO health insurance plan',
  'common_complaints': ['High premiums and deductibles',
   'Denied claims and billing errors',
   'Inefficient customer service with long wait times',
   'Limited provider networks, especially in rural areas',
   'Pre-authorization requirements causing treatment delays'],
  'major_positives': ['Comprehensive coverage options',
   'Access to a wide network of providers',
   'Preventive care services and wellness programs',
   'Partnership with CVS Health offering additional member benefits'],
  'summary': "Aetna's Silver PPO health insurance plan offers comprehensive coverage with access to a broad provider network and preventive care services. However, customers have reported high premiums and deductibles, denied claims, billing errors, and inefficient customer service. Additionally, some members have experienced limited provider options in certain areas and delays due to pre-authorization requirements.

In [90]:
# Building UI

In [93]:
import sys
%pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [95]:
import asyncio
import json
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output


In [97]:
async def analyze_product(company: str, product: str):
    # 1) Research using insurance_agent
    query = (
        f"Analyze customer feedback for {company}'s {product}. "
        "Find common complaints, major positives, and overall summary."
    )
    result = await Runner.run(insurance_agent, query)
    raw_output = result.final_output

    # Extract JSON from any surrounding text
    start = raw_output.find("{")
    end = raw_output.rfind("}") + 1
    research_data = json.loads(raw_output[start:end])

    # 2) SWOT using swot_agent
    swot_prompt = (
        "Create a SWOT analysis from the following JSON data:\n\n"
        f"{json.dumps(research_data, indent=2)}"
    )
    swot_result = await Runner.run(swot_agent, swot_prompt)
    swot_raw = swot_result.final_output

    start = swot_raw.find("{")
    end = swot_raw.rfind("}") + 1
    swot_data = json.loads(swot_raw[start:end])

    return {
        "research": research_data,
        "swot": swot_data,
    }


In [99]:
# Input fields
company_input = widgets.Text(
    value="Aetna",
    placeholder="Enter insurance company (e.g., Aetna)",
    description="Company:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width="500px"),
)

product_input = widgets.Text(
    value="Silver PPO health insurance plan",
    placeholder="Enter product name",
    description="Product:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width="500px"),
)

run_button = widgets.Button(
    description="Run Analysis",
    button_style="primary",  # 'primary', 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to analyze this product",
)

output_area = widgets.Output()


In [101]:
async def run_analysis_for_inputs():
    company = company_input.value.strip()
    product = product_input.value.strip()

    if not company or not product:
        with output_area:
            clear_output()
            display(Markdown("⚠️ **Please enter both company and product.**"))
        return

    with output_area:
        clear_output()
        display(Markdown(f"### Running analysis for **{company} – {product}**...\nThis may take a few moments."))

    # Call your pipeline
    analysis = await analyze_product(company, product)

    research = analysis["research"]
    swot = analysis["swot"]

    # Nicely formatted output
    with output_area:
        clear_output()

        display(Markdown(f"## 📊 Analysis for **{company} – {product}**"))

        # Research summary
        display(Markdown("### 📝 Summary"))
        display(Markdown(research.get("summary", "_No summary available._")))

        # Complaints
        complaints = research.get("common_complaints", [])
        positives = research.get("major_positives", [])

        if complaints:
            display(Markdown("### ❌ Common Complaints"))
            display(Markdown("\n".join(f"- {c}" for c in complaints)))

        if positives:
            display(Markdown("### ✅ Major Positives"))
            display(Markdown("\n".join(f"- {p}" for p in positives)))

        # SWOT
        display(Markdown("## 🧠 SWOT Analysis"))

        def bullet_list(title, items):
            if not items:
                return
            display(Markdown(f"### {title}"))
            display(Markdown("\n".join(f"- {item}" for item in items)))

        bullet_list("💪 Strengths", swot.get("strengths", []))
        bullet_list("⚠️ Weaknesses", swot.get("weaknesses", []))
        bullet_list("📈 Opportunities", swot.get("opportunities", []))
        bullet_list("⚡ Threats", swot.get("threats", []))


def on_run_button_clicked(b):
    # Schedule the async task
    asyncio.create_task(run_analysis_for_inputs())

run_button.on_click(on_run_button_clicked)


In [123]:
ui = widgets.VBox([
    widgets.HTML("<h2> Insurance Product Analyzer</h2>"),
    company_input,
    product_input,
    run_button,
    output_area,
])

display(ui)
